<a href="https://colab.research.google.com/github/alectomas/cms-datocms/blob/main/AI_Calendar.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install icalendar

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 112.1/112.1 kB 2.5 MB/s eta 0:00:00


In [ ]:
import datetime
from icalendar import Calendar
import pytz
from google.colab import files

# Function to parse .ics file
def parse_ics(file_path):
    with open(file_path, 'rb') as f:
        gcal = Calendar.from_ical(f.read())
    return gcal

# Function to filter events
def filter_events(calendar, start_date, end_date):
    events = []
    for component in calendar.walk('vevent'):
        event_start = component.get('dtstart').dt

        # Ensure event_start is a datetime object and has timezone info
        if not isinstance(event_start, datetime.datetime):
            event_start = datetime.datetime.combine(event_start, datetime.time())
        if event_start.tzinfo is None:
            event_start = event_start.replace(tzinfo=pytz.UTC)

        if start_date <= event_start <= end_date:
            events.append(component)
    return events


# Function to find available time slots
def find_available_slots(events, start_date, end_date):
    weekdays = [start_date + datetime.timedelta(days=x) for x in range((end_date - start_date).days + 1) if (start_date + datetime.timedelta(days=x)).weekday() < 5]
    slots = {day: [(datetime.datetime.combine(day, datetime.time(hour=9)), datetime.datetime.combine(day, datetime.time(hour=17)))] for day in weekdays}

    for event in events:
        start = event.get('dtstart').dt
        end = event.get('dtend').dt
        day = start.date()
        if day in slots:
            slots[day] = [slot for slot in slots[day] if not (slot[0] < end and slot[1] > start)]

    return slots

In [ ]:
# Function to generate markdown content
def generate_markdown(events, available_slots):
    md_content = "# Events Summary\n\n"
    for event in events:
        summary = event.get('summary')
        start = event.get('dtstart').dt
        end = event.get('dtend').dt
        attendees = event.get('attendee')
        description = event.get('description')

        md_content += f"## {summary}\n- **Start**: {start}\n- **End**: {end}\n"

        if attendees:
            # Check if attendees is a list or a single item
            if isinstance(attendees, list):
                attendee_names = []
                for attendee in attendees:
                    if isinstance(attendee, str):
                        attendee_names.append(attendee)
                    else:
                        attendee_names.append(attendee.params.get('CN', 'Unknown'))
                md_content += "- **Attendees**: " + ', '.join(attendee_names) + "\n"
            else:
                # Handle the case where there's only one attendee
                if isinstance(attendees, str):
                    md_content += f"- **Attendees**: {attendees}\n"
                else:
                    md_content += f"- **Attendees**: {attendees.params.get('CN', 'Unknown')}\n"

        if description:
            md_content += f"- **Description**: {description}\n"
        md_content += "\n"

    md_content += "# Available Time Slots\n\n"
    for day, slots in available_slots.items():
        md_content += f"## {day.strftime('%Y-%m-%d')}\n"
        for start, end in slots:
            md_content += f"- From {start.time()} to {end.time()}\n"
        md_content += "\n"

    return md_content



# Main execution
file_path = 'your_calendar_file.ics'  # Replace with your .ics file path
calendar = parse_ics(file_path)

# Define the date ranges
utc = pytz.UTC
today = datetime.datetime.now().astimezone(utc)
past_3_weeks = today - datetime.timedelta(weeks=3)
next_3_weeks = today + datetime.timedelta(weeks=3)
next_2_weeks = today + datetime.timedelta(weeks=2)

# Filter events and find available slots
filtered_events = filter_events(calendar, past_3_weeks, next_3_weeks)
available_slots = find_available_slots(filtered_events, today, next_2_weeks)

# Generate markdown content
markdown_content = generate_markdown(filtered_events, available_slots)

# Save to a markdown file
with open('events_summary.md', 'w') as md_file:
    md_file.write(markdown_content)




In [ ]:
# Save to a markdown file
with open('events_summary.md', 'w') as md_file:
    md_file.write(markdown_content)

# Read and display the contents of the file
with open('events_summary.md', 'r') as md_file:
    print(md_file.read())

# Download the file
files.download('events_summary.md')

# json=generate_jsonl(filtered_events, available_slots)
# print(json)


# Events Summary

## monashees private dinner
- **Start**: 2023-11-14 22:30:00+00:00
- **End**: 2023-11-15 01:30:00+00:00
- **Description**: Get up-to-date information at: https://lu.ma/monasheesdinner

Address:
Vitacura

Estás cordialmente invitado a una cena que se llevará a cabo el martes 14 de noviembre a las 7:30 pm.

Se trata de una cena informal con otros founders y el equipo de monashees, con el propósito de conocernos y expandir nuestra red en Santiago.

Por favor, confirma tu asistencia o declina la invitación antes del 3 de noviembre, para que podamos calcular el número de lugares necesarios.

Este es un evento privado y solo por invitación.

¡Esperamos verte allí!

monashees

## Alec Dickinson and Isaac de la Peña
- **Start**: 2023-11-15 13:00:00+00:00
- **End**: 2023-11-15 13:30:00+00:00
- **Attendees**: mailto:isaac@conexo.vc, mailto:carlos@42x.ai, mailto:alec@42x.ai, mailto:emilio@42x.ai
- **Description**: Event Name: 30 Minute Meeting

Location: This is a Google Meet we

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>